In [ ]:

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing libraries
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pwd
# !kaggle datasets list
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
/content
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.9/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.9/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip new-plant-diseases-dataset.zip

unzip:  cannot find or open new-plant-diseases-dataset.zip, new-plant-diseases-dataset.zip.zip or new-plant-diseases-dataset.zip.ZIP.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.applications import InceptionV3
from glob import glob
from sklearn.model_selection import train_test_split

In [ ]:
train = "/content/plant/d/train"
test= "test/test"
valid = "/content/plant/d/valid"

In [ ]:
img = load_img("./plant/d/train/Apple___healthy/0055dd26-23a7-4415-ac61-e0b44ebfaf80___RS_HL 5672.JPG")
plt.imshow(img)
plt.axis("off")
plt.title("Sample Anhinga Image")
plt.show()

plt.figure()

FileNotFoundError: ignored

In [ ]:
className = glob(train + "/*")
NumberofClass = len(className)
print("NumberofClass:", NumberofClass)
className

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    rescale=1/255
        
    )
# datagen.mean = [123.68, 116.779, 103.939]
train_it = datagen.flow_from_directory(train,                           
    class_mode='categorical', batch_size=32, target_size=(224, 224))
test_it = datagen.flow_from_directory(valid,
        class_mode='categorical', batch_size=32, target_size=(224, 224))


In [ ]:
target_names=train_it.class_indices
names=[]
print('len(target_names',len(target_names))
dc={};
for i,j in target_names.items():
  names.append(i);
  dc[j]=i;
for i in names:
  print(i)
# print(target_names);print()

In [ ]:
# define cnn model
from keras.applications.vgg16 import VGG16
from keras.models import Model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(38, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model

In [ ]:
model = define_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
%%time
mo = keras.models.load_model("vgg.h5")
epochsteps=len(train_it);epochs =10;valsteps=len(test_it)
# fit model

history = mo.fit(train_it, steps_per_epoch=epochsteps,
                    validation_data=test_it, validation_steps=valsteps, epochs=10, verbose=1)
_, acc = mo.evaluate(test_it, steps=75, verbose=1)
print('accuracy',acc * 100.0)

In [ ]:
model.save("vgg1.h5");

In [ ]:
t = datagen.flow_from_directory(
                "/content/test",
                target_size=(224,224),
                batch_size=32,
                color_mode="rgb",
                class_mode="categorical"
                )

In [ ]:

mo = keras.models.load_model("vgg.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
def predict_image(filename, model):
    img_ = image.load_img(filename, target_size=(224, 224))
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.
    print("ORIGINAL NAME :",filename[19:-5])
    
    prediction = model.predict(img_processed)
    
    index = np.argmax(prediction)
    print("predicted name is : ",dc[index])
    d={}
    j=0;
    for i in prediction[0]:
      d[i]=dc[j]
      j+=1
    d=sorted(d.items())
    print("ACCURACY: ",d[-1][0]*100,"%")

    p=prediction[0][0]
    
    plt.imshow(img_array)

In [ ]:
predict_image("/content/test/test/TomatoYellowCurlVirus5.JPG",mo);

In [ ]:
predict_image("/content/test/test/AppleScab2.JPG",mo);

In [ ]:
predict_image("/content/test/test/PotatoEarlyBlight3.JPG",mo);